In [1]:
import numpy as np
import pandas as pd
import altair as alt
import KDE_prepare_data
import KDE_utils
import correlations
from KDE_settings import Config
from scipy import stats
alt.renderers.enable('altair_viewer')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## MP particle size correlations

*This calculates probability density distributions for MP sizes. These are used to estimate the abundances of MP of specific size ranges, wich can then be investigated for correlations to environmental paramteres such as TOC or sediment grain size properties.*
**Before running, check the computational parameters in KDE_settings.py**

In [2]:
# What happened so far: DB extract and blank procedure. Now import reulting MP data from csv
pdd_MP = pd.read_csv('../csv/env_MP_clean_list_SchleiSediments.csv', index_col=0)

# Also import sediment data (sediment frequencies per size bin from master sizer export)
sed_size_freqs = pd.read_csv('/home/nibor/ownCloud/microSCHLEI/Sediment_K/Sediment analysis/KGA/KGA_IOW_Schlei_export/Enders_export_10µm_linear_noR_RL.csv')

In [3]:
# Get the binning structure of the imported sediment data and optionally rebin it (make binning coarser) for faster computation
sed_size_freqs, sed_x_d = KDE_prepare_data.sediment_preps(sed_size_freqs, rebinning=False)

In [4]:
# ...some data wrangling to prepare particle domain data and sample domain data for MP and combine with certain sediment aggregates.
sdd_MP = KDE_prepare_data.aggregate_SDD(pdd_MP)
sdd_MP_sed = KDE_prepare_data.add_sediment(sdd_MP)
pdd_sdd_MP = KDE_prepare_data.sdd2pdd(sdd_MP_sed, pdd_MP)

In [5]:
# KDEs for probability of finding a MP particle in a specific size bin are calculated.
size_pdfs = KDE_utils.per_sample_kde(pdd_sdd_MP, x_d = sed_x_d)

In [6]:
# MP abundances per size bin are calculated from the KDEs and the total MP concentration in the samples.
df_range_conc = KDE_utils.make_range_conc(size_pdfs, sdd_MP_sed)

In [7]:
# MP and sediment abundances per size bin are equalised (only samples and bins contained in both of the are kept).
# They are then melted and merged into MPsedMelted.
df_range_conc, sed_size_freqs, MPsedMelted = KDE_prepare_data.equalise_MP_and_Sed(df_range_conc, sed_size_freqs)

In [12]:
brush1 = alt.selection(type="interval", encodings=['x'])
brush2 = alt.selection(type="interval", encodings=['x'])


scatter = alt.Chart(MPsedMelted).mark_point().encode(
    x = 'freq_sum:Q',
    y = 'conc_sum:Q',
    tooltip = 'sample_name:N'
).transform_filter(
    brush1 | brush2
).transform_aggregate(
    conc_sum = 'sum(conc)',
    freq_sum = 'sum(freq)',
    sample_name = 'values(sample)',
    groupby = ['sample']
# ).transform_calculate(
#     b1l='(isDefined(brush1.x) ? (brush1.x[0]) : 1)',
#     b1u='(isDefined(brush1.x) ? (brush1.x[1]) : 1)',
#     b2l='(isDefined(brush2.x) ? (brush2.x[0]) : 1)',
#     b2u='(isDefined(brush2.x) ? (brush2.x[1]) : 1)',
# ).transform_window(
#     conc_sum = 'sum(conc)',
#     frame = [brush1.x[0],brush1.x[1]],
#     groupby=['sample']
# ).transform_window(
#     freq_sum = 'sum(freq)',
#     frame = [brush2.x[0],brush2.x[1]],
#     groupby=['sample']
)


RegLine = scatter.transform_regression(
    'freq_sum', 'conc_sum', method="linear",
).mark_line(
    color="red"
)


RegParams = scatter.transform_regression(
    'freq_sum', 'conc_sum', method="linear", params=True
).mark_text(align='left', lineBreak='\n').encode(
    x=alt.value(120),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"r² = " + round(datum.rSquared * 100)/100 + \
    "      y = " + round(datum.coef[1] * 10)/10 + "x" + " + " + round(datum.coef[0] * 10)/10'
)


MP = alt.Chart(MPsedMelted).mark_line().encode(
    x = alt.X('lower', scale=alt.Scale(type='linear')),
    y = 'mean(conc)'
).add_selection(
    brush1
).properties(
    height = 100
)


sed = alt.Chart(MPsedMelted).mark_line().encode(
    x = alt.X('lower', scale=alt.Scale(type='linear')),
    y = 'mean(freq)'
).add_selection(
    brush2
).properties(
    height = 100
)


(scatter + RegLine + RegParams) & alt.vconcat(MP, sed)

Displaying chart at http://localhost:15411/

In [2]:
import altair as alt
from vega_datasets import data
cars = data.cars.url

brush1 = alt.selection(type="interval", encodings=['x'])
brush2 = alt.selection(type="interval", encodings=['x'])


scatter = alt.Chart(cars).mark_point().encode(
    x = 'HP_sum:Q',
    y = 'Dis_sum:Q',
    tooltip = 'Origin:N'
).transform_filter(  # Ok, I can filter the whole data set, but that always acts on both variables (HP and displacement) together... -> not what I want.
    brush1 | brush2
).transform_aggregate(
    Dis_sum = 'sum(Displacement)',
    HP_sum = 'sum(Horsepower)',
    groupby = ['Origin', 'HP_sum']
# ).transform_calculate(  # Can I extract the selection boundaries like that? And if yes: how can I use these extracts to calculate the aggregationsof HP and displacement?
#     b1_lower='(isDefined(brush1.x) ? (brush1.x[0]) : 1)',
#     b1_upper='(isDefined(brush1.x) ? (brush1.x[1]) : 1)',
#     b2_lower='(isDefined(brush2.x) ? (brush2.x[0]) : 1)',
#     b2_upper='(isDefined(brush2.x) ? (brush2.x[1]) : 1)',
# ).transform_window(  # Maybe instead of calculate I can use two window transforms...??
#     conc_sum = 'sum(conc)',
#     frame = [brush1.x[0],brush1.x[1]],  # This will not work, as it sets the frame relative (back- and foreward) to each datum (i.e. sliding window), I need it to correspond to the entire data set
#     groupby=['sample']
# ).transform_window(
#     freq_sum = 'sum(freq)',
#     frame = [brush2.x[0],brush2.x[1]],  # ...same problem here
#     groupby=['sample']
)

range_sel1 = alt.Chart(cars).mark_line().encode(
    x = 'Year:T',
    y = 'mean(Horsepower):Q'
).add_selection(
    brush1
).properties(
    height = 100
)

range_sel2 = alt.Chart(cars).mark_line().encode(
    x = 'Year:T',
    y = 'mean(Displacement):Q'
).add_selection(
    brush2
).properties(
    height = 100
)


scatter & range_sel1 & range_sel2

alt.VConcatChart(...)

In [8]:
# Sum abundances in size bins for all possible combinations. Basically create all possible rebinnings and stack them into one DF
MPext = KDE_prepare_data.combination_sums(df_range_conc)
SEDext= KDE_prepare_data.combination_sums(sed_size_freqs)

In [25]:
# Calculate a correlation matrix containing Pearson correlation coefficients for all combinations of any original or summed bins of MP and sediments.
corrMat = np.corrcoef(MPext,SEDext)
corrMat = corrMat[:len(MPext), len(SEDext):]  # only take upper right quadrant of correlation matrix

corrMatDF = pd.DataFrame(corrMat, index=MPext.index, columns=SEDext.index)  # turn np array into df
corrMatDF.rename('MP_{}'.format, axis=0, inplace=True)  # add a prefix for 'MP' to each row label
corrMatDF.rename('SED_{}'.format, axis=1, inplace=True)  # add a prefix for 'sediment' to each column label

In [142]:
# Run a cross-correlation for each sample between MP and sediment size distribution curve shapes.
crosscorr_results = pd.DataFrame()

for label, content in df_range_conc.items():
    datax = content
    datay = sed_size_freqs[label]
    #best = correlations.crosscorr(datax, datay)
        
#     lags = range(-int(len(datax)/2), int(len(datax)/2)+1)
    lags = range(-50,0)
    df_r = pd.DataFrame(lags, columns = ['shifted'])
    
#     r_pear = [datax.corr(datay.shift(lag)) for lag in lags]
#     df_r['pearson_r'] = r_pear
    r_spear = [stats.spearmanr(np.array(datax),np.array(datay.shift(lag)), nan_policy = 'omit') for lag in lags]
    df_r['spearman_r'] = r_spear
    
    
    best = df_r.copy().loc[df_r['spearman_r'] == df_r['spearman_r'].max()]
    best['Sample'] = label
    crosscorr_results = crosscorr_results.append(best)

In [17]:
# Calculate correlation between all size bins (or their summed up combinations) of MP abundances and a scalar predictor.
# Possible predictors are: 'TOC', 'Hg', 'Dist_WWTP' or different sediment grain size properties (D50, below 63, etc.). See column labels of sdd_MP_sed for a full list.
bestLower, bestUpper, df_r = correlations.predictorcorr(df_range_conc, sdd_MP_sed.set_index('Sample'), 'TOC')

    lower_size upper_size         r         p990,        2000]                


In [ ]:
# Heatmap correlation coefficients from the single predictor correlation
alt.renderers.enable('html')

alt.Chart(df_r).mark_point().encode(
    x='lower_size',
    y='upper_size',
    color=alt.Color("r", scale=alt.Scale(domain=[0.5, 0.6])),
    tooltip=['r', 'lower_size', 'upper_size']
).properties(
    width=800,
    height=800
)

In [ ]:
#####USED only for testing external function inside NB: #####
step = (Config.upper_size_limit - Config.lower_size_limit) / Config.kde_steps
df_r = pd.DataFrame(columns=['lower_size', 'upper_size', 'r', 'p'])

for i in [10]:
    for j in [1000]:
        size_sum = size_pdfs.loc[(size_pdfs.x_d >= i) & (size_pdfs.x_d < j)].sum()
        size_sum.drop('x_d', inplace=True)
        range_prob = size_sum * step
        range_conc = range_prob * sdd_MP_sed.set_index('Sample').Concentration

        r = stats.pearsonr(range_conc, sdd_MP_sed.set_index('Sample').TOC)
        df_r.loc[len(df_r)] = [i, j, r[0], r[1]]
        print(f'Correlating TOC with size range            [{i},        {j}]                ', end="\r", flush=True)

print(df_r.loc[df_r.r == df_r.r.max()])
bestLower, bestUpper = df_r.loc[df_r.r == df_r.r.max()].iloc[0, 0:2]

In [ ]:
# Property-property-plot with MP against single predictor incl. reg line 
kd = alt.Chart(pd.DataFrame([range_conc, sdd_MP_sed.set_index('Sample').TOC]).T.reset_index()).mark_point().encode(
    x='TOC',
    y='Unnamed 0',
    color='index',
    tooltip='index'
)

Reg_Line = kd.transform_regression('TOC', 'Unnamed 0', method="linear",
                                  ).mark_line(color="red")

Reg_Params = kd.transform_regression('TOC', 'Unnamed 0', method="linear", params=True
                                    ).mark_text(align='left', lineBreak='\n').encode(
        x=alt.value(120),  # pixels from left
        y=alt.value(20),  # pixels from top
        text='params:N'
    ).transform_calculate(
        params='"r² = " + round(datum.rSquared * 100)/100 + \
    "      y = " + round(datum.coef[0] * 10)/10 + " + e ^ (" + round(datum.coef[1] * 10000)/10000 + "x" + ")" + \n + " "'
    )

kd + Reg_Line + Reg_Params

In [59]:
daf = sed_size_freqs.reset_index().melt(id_vars='index', value_name='value')
daf['size'] = daf['index'].str.split('_').str[0]
daf['category'] = 'sediment'

daff = df_range_conc.reset_index().melt(id_vars='index', value_name='value')
daff['size'] = daf['index'].str.split('_').str[0]
daff['category'] = 'MP'

df = pd.concat([daf, daff]).drop(columns=['index'])
# df = daf.merge(daff, on =['sample', 'size', 'index']).drop(columns=['index'])

In [120]:
input_dropdown = alt.binding_select(options=list(df['sample'].unique()), name='select sample ')
selection = alt.selection_single(fields=['sample'],
                                 bind=input_dropdown,
                                 #init={'select_': 'Schlei_S8'}
                                )

base = alt.Chart(df).mark_line().encode(
    x=alt.X('size:Q'),
    y=alt.Y('value:Q'),
    color='category',
#     shape='category',
#     detail='category'
# ).transform_window(
#     cuml='sum(value)',
#     groupby=['sample']
).transform_filter(
    alt.FieldRangePredicate(field='size', range=[50, 990])
# ).properties(
#     width=180,
#     height=180
# ).facet(
#     facet='sample',
#     columns=6
# ).resolve_scale(
#     y='independent'
)


mps = base.mark_line(color='blue').transform_filter(
    alt.FieldEqualPredicate(field='category', equal='MP')
)
seds = base.mark_line(color='yellow').transform_filter(
    alt.FieldEqualPredicate(field='category', equal='sediment')
)

alt.layer(mps, seds).resolve_scale(
    y = 'independent'
).add_selection(
    selection
).transform_filter(
    selection
)


Displaying chart at http://localhost:23565/

In [151]:
base = alt.Chart(df).encode(
    alt.X('size:Q', axis=alt.Axis(title=None))
).properties(
    width=180,
    height=180
)

mps = base.mark_line(stroke='#57A44C', interpolate='monotone').encode(
    alt.Y('MP:Q', axis=alt.Axis(title='MP', titleColor='#57A44C'))
)

seds = base.mark_line(stroke='#5276A7', interpolate='monotone').encode(
    alt.Y('sediment:Q', axis=alt.Axis(title='sediment', titleColor='#5276A7'))
)

alt.layer(mps, seds).resolve_scale(
    y = 'independent'
).properties(
    width=180,
    height=180
).facet(
    facet='sample',
    columns=6
).resolve_scale(
    y='independent'
)

Displaying chart at http://localhost:23565/

In [177]:
df = daf.shift(100).merge(daff, on =['sample', 'size', 'index']).drop(columns=['index'])
alt.Chart(df).mark_point().encode(
    x=alt.X('value_x', scale=alt.Scale(type='linear')),
    y=alt.X('value_y', scale=alt.Scale(type='linear')),
    color=alt.Color('size:Q', scale=alt.Scale(scheme="viridis"))
).properties(
    width=150,
    height=150
).facet(
    facet='sample',
    columns=6
).resolve_scale(
    y='independent',
    x='independent'
)

Displaying chart at http://localhost:23565/

In [15]:
data = pd.DataFrame({
    'x': range(9),
    'y': [1, 2, 3, 4, 5, 4, 3, 2, 1]
})

brush = alt.selection_interval(name="brush", encodings=['x'])
alt.Chart(data).mark_line().add_selection(brush).transform_calculate(
    scaled_by_brush_width='datum.y * (isDefined(brush.x) ? (brush.x[1] - brush.x[0]) : 1)'
).encode(
    x='x:Q',
    y='scaled_by_brush_width:Q'
)

Displaying chart at http://localhost:19233/